In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")

In [ ]:
df.head()

COLUMNS VALUE REPRESENTING:
* 1: age
* 2: sex(1 = male, 0 = female)
* 3: chest pain type (4 values)
* 4: resting blood pressure
* 5: serum cholestoral in mg/dl
* 6: fasting blood sugar > 120 mg/dl
* 7: resting electrocardiographic results (values 0,1,2)
* 8: maximum heart rate achieved
* 9: exercise induced angina
* 10: oldpeak = ST depression induced by exercise relative to rest
* 11: the slope of the peak exercise ST segment
* 12: number of major vessels (0-3) colored by flourosopy
* 13: thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

In [ ]:
#CHCEKING VALUT TYPE AND NOT-NULL VALUES
df.info()

In [ ]:
# CHECKING MEAN STANDARD DEVIATION AND OTHER INFORMATIONS FOR COLUMNS
df.describe()

In [ ]:
#CHECK COVARIANCE RELATION BETWEEN COLUMNS 
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (15,10))
#df.corr()
sns.heatmap(df.corr(),annot = True,linecolor = "RED")
plt.show()

FROM THIS CORRELATION GRAPH WE CAN SEE IT IS SHOWING GOOD CORRELATION BETWEEN TARGET AND CHEST PAIN AND TARGET AND THALACH

NOW LETS CHECK HOW HEART DISEASE DEPENDS ON AGE. AS FAR ON OUR GOOGLE KNOWLEDGE WE ASSUME THAT OLD AGES ARE MORE PRONE
TO GET HEART DISEASE.

In [ ]:
plt.scatter(df['target'],df['age'])
plt.xlabel("Target")
plt.ylabel("Age")
plt.show()

As 0 represent no heart disease and 1 represent heart disease. And by the upper graph it seems all the age group are having 
heart disease patients.
Lets plot distribution plot to see how many patients falls in which age groups.

In [ ]:
# Filtering patients
patients = df[df['target'] == 1]
sns.distplot(patients['age'],color = "RED")

Age group 50-60 having more patients than any other age groups.

In [ ]:
plt.figure(figsize = (10,5))
sns.violinplot( df["sex"] ,df["target"], color="RED")
plt.title("Rleation between gender and target")
plt.show()
#sns.distplot( df["target"] , color="red")
#sns.plt.legend()
#plt.scatter(df['sex'],df['target'])


As 0 shows female on x axis and 1 shows heart disease on y axis, and it is visible that there are high probability distribution for female means more female patient probability. Lets confirm it manually.

In [ ]:
print(df['sex'].value_counts())
female = df[df['sex'] == 0]
female_patients = female[female['target'] == 1]
male = df[df['sex'] == 1]
male_patients = male[male['target'] == 1]
print("Probability for women having heart disease" + str(len(female_patients)/len(female)))
print("Probability for men having heart disease" + str(len(male_patients)/len(male)))

In [ ]:
plt.figure(figsize = (10,5))
sns.swarmplot(df['target'],df['cp'], size = 8)
plt.xlabel("Target")
plt.ylabel("Chest Pain")
plt.title("Relation between chest pain and target")
plt.show()

In [ ]:
plt.figure(figsize = (10,5))
sns.swarmplot(df['target'],df['trestbps'], size = 8)
plt.xlabel("Target")
plt.ylabel("Blood Pressure on Rest Time")
plt.title("Relation between Rest Time Blood Pressure and target")
plt.show()

RELATION BETWEEN A PERSON'S CHOLESTROL AND HEART DISEASE.

In [ ]:
plt.figure(figsize = (10,5))
sns.swarmplot(df['target'],df['chol'], size = 8)
plt.title("Relation between cholestral and target")
plt.show()

This function is similar to stripplot(), but the points are adjusted (only along the categorical axis) so that they don’t overlap. This gives a better representation of the distribution of values, but it does not scale well to large numbers of observations. This style of plot is sometimes called a “beeswarm”.

In [ ]:
pd.crosstab(df['fbs'],df['target']).plot(kind="bar",figsize=(10,7),color=['RED','BLUE' ])
#plt.xlabel('Target')
#plt.ylabel('Frequency of Disease or Not')
plt.show()

In [ ]:
pd.crosstab(df['fbs'],df['target'])

In [ ]:
pd.crosstab(df['restecg'],df['target']).plot(kind="bar",figsize=(10,7),color=['yellow','purple' ])
#plt.xlabel("")


In [ ]:
plt.figure(figsize = (10,5))
sns.swarmplot(df['target'],df['thalach'], size = 8)
plt.title("Relation between maximum heart rate achieved and target")
plt.show()

ACCORDING TO UCI REPOSITORY INFORMATION

categorical variable = 'sex','chest_pain_type','fasting_blood_sugar','rest_ecg','exercise_induced_angina','st_slope','num_major_vessels','thalassemia','target'
numerical variable = 'age','resting_blood_pressure','cholesterol','max_heart_rate_achieved','st_depression','target' Because they are categorial values but represent in int modle is not able to work fairly, So changing it in dummie variables.

In [ ]:

X = df[['sex','cp','fbs','restecg','exang','slope','ca','thal']].astype(object)
dummies = pd.get_dummies(X)
dummies.info()

In [ ]:
df.drop(['sex','cp','fbs','restecg','exang','slope','ca','thal'], inplace = True,axis = 1)

In [ ]:
df = pd.concat([df,dummies] ,axis = 1)

In [ ]:
df

In [ ]:
#CHANGING NUMERICAL VALUES TO THEIR STANDARD VALUE
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
nm = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
df[nm] = scale.fit_transform(df[nm])

In [ ]:
df

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import train_test_split
y = df.target
df.drop('target',inplace = True,axis = 1)
print(y.shape)
print(df.shape)
x_train, x_test, y_train, y_test = train_test_split(df,y,test_size = 0.2,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
lr.fit(x_train,y_train)
pred = lr.predict(x_test)
train_acc = accuracy_score(y_train,lr.predict(x_train))
acc = accuracy_score(y_test,pred)
print("Training_Accuracy" + str(train_acc))
print("Test_Accuracy" + str(acc))

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)
pred1 = nb.predict(x_test)
acc1 = accuracy_score(y_test,pred1)
train_acc1 = accuracy_score(y_train,nb.predict(x_train))
print("Training_Accuracy" + str(train_acc1))
print("Test_Accuracy" + str(acc1))

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train, y_train)
pred2 = svm.predict(x_test)
acc2 = accuracy_score(y_test,pred2)
acc2

STANDARD SCALING IS EFFECTING SVC ACCURACY AND IT INCREASES.

Try some hyperparameter tuning and see its effect on accuracy.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {"C": np.logspace(-4, 4, 20),
          "solver": ["liblinear"]}


grid_search_cv = GridSearchCV(lr, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=5, iid=True)
grid_search_cv.fit(x_train, y_train)

In [ ]:
grid_search_cv.best_estimator_

In [ ]:
log_reg = LogisticRegression(C=0.23357214690901212, 
                             solver='liblinear')

log_reg.fit(x_train, y_train)


In [ ]:
pred5 = log_reg.predict(x_test)
acc5 = accuracy_score(y_test,pred5)
acc5

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred5))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,pred5)

In [ ]:
cm

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000,random_state = 42)
rf.fit(x_train,y_train)
rf_pred = rf.predict(x_test)
print(accuracy_score(y_test,rf_pred))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [ ]:
param_dist = {"max_depth": [3, None],"max_features": sp_randint(1, x_train.shape[1]),"min_samples_split": sp_randint(2, 11),"bootstrap": [True, False],"n_estimators": sp_randint(100, 500)}

In [ ]:
random_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                   n_iter=10, cv=5, iid=False, random_state=42)
random_search.fit(x_train, y_train)

In [ ]:
print(random_search.best_params_)

In [ ]:
rf1 = RandomForestClassifier(bootstrap = False, max_depth = None, max_features = 3, min_samples_split = 7,n_estimators = 408)
rf1.fit(x_train,y_train)
rf1_pred = rf1.predict(x_test)
print(accuracy_score(y_test,rf1_pred))

**CONCLUSION**
* When we are changing catogorical values with function get dummies then it help us to increase accuracy, and after standerising our numeric variable SVC algorithm is working better than earlier.As Logistic Regression is showing best accuracy for this accuracy because it is a decision making problem.